To download the dataset 
```shell
$ mkdir linkage
$ cd linkage/
$ curl -L -o donation.zip https://bit.ly/1Aoywaq
$ unzip donation.zip
$ unzip 'block_*.zip'
```

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Question 1

Load the data

In [41]:
path = 'data/linkage/block*.csv'
df = spark.read.csv(path, header=True, nullValue='?', inferSchema=True)

Cache the DataFrame

["Explaining the mechanics of Spark caching"](https://luminousmen.com/post/explaining-the-mechanics-of-spark-caching) -- great article explaining why and how Spark DataFrames are cached

The first action run after `df.cache()` will take its time (including overhead of caching). All subsequent actions will be fast it seems.

In [42]:
df.cache()

%timeit -n 1 -r 1 df.count() # slow
%timeit -n 1 -r 1 df.count() # fast

24/11/08 00:57:45 WARN CacheManager: Asked to cache already cached data.


234 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
63.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Create summary statistics for matches and non-matches

In [49]:
def pivot(spark_df):
    return spark.createDataFrame(
        spark_df.toPandas().set_index('summary').T.reset_index()
    )

match_stats = pivot(df.where("is_match = True").describe())
miss_stats = pivot(df.where("is_match = False").describe())

match_stats.show()
miss_stats.show()

+------------+-----+------------------+--------------------+---+-----+
|       index|count|              mean|              stddev|min|  max|
+------------+-----+------------------+--------------------+---+-----+
|        id_1|20931| 34575.72117911232|   21950.31285196913|  5|99946|
|        id_2|20931| 51259.95939037791|   24345.73345377519|  6|99996|
|cmp_fname_c1|20922|0.9973163859635038| 0.03650667584833679|0.0|  1.0|
|cmp_fname_c2| 1333|0.9898900320318176| 0.08251973727615237|0.0|  1.0|
|cmp_lname_c1|20931|0.9970152595958817|0.043118807533945126|0.0|  1.0|
|cmp_lname_c2|  475| 0.969370167843852| 0.15345280740388917|0.0|  1.0|
|     cmp_sex|20931| 0.987291577086618| 0.11201570591216435|  0|    1|
|      cmp_bd|20925|0.9970848267622461| 0.05391487659807981|  0|    1|
|      cmp_bm|20925|0.9979450418160095|0.045286127452170664|  0|    1|
|      cmp_by|20925|0.9961290322580645| 0.06209804856731055|  0|    1|
|     cmp_plz|20902|0.9584250310975027| 0.19962063345931919|  0|    1|
+-----

Compute how much each field differs for matches versus non-matches

### Question 2

First select relevant features

What makes a good feature?
- Has different values for matches and non-matches
- Available most of the time (not null for most instances)

In [51]:
match_stats.createOrReplaceTempView('match_stats')
miss_stats.createOrReplaceTempView('miss_stats')

query = '''
    select A.index, A.count + B.count as occurances, A.mean - B.mean as delta
    from match_stats A 
        inner join miss_stats B on A.index = B.index
    where A.index not in ('id_1', 'id_2')
    order by delta desc;
'''
spark.sql(query).show()

+------------+----------+--------------------+
|       index|occurances|               delta|
+------------+----------+--------------------+
|     cmp_plz| 5736289.0|  0.9563812499852176|
|cmp_lname_c2|    2464.0|  0.8064147192926266|
|      cmp_by| 5748337.0|  0.7762059675300512|
|      cmp_bd| 5748337.0|   0.775442311783404|
|cmp_lname_c1| 5749132.0|  0.6838772482594513|
|      cmp_bm| 5748337.0|  0.5109496938298685|
|cmp_fname_c1| 5748125.0|  0.2854529057459947|
|cmp_fname_c2|  103698.0| 0.09104268062280174|
|     cmp_sex| 5749132.0|0.032408185250332844|
+------------+----------+--------------------+



In [52]:
features = ["cmp_plz", "cmp_by", "cmp_bd", "cmp_lname_c1", "cmp_bm"]

In [ ]:
from pyspark.sql.functions import expr

scored = df.fillna(0).\
    withColumn('score', expr(" + ".join(features))).\
    select('score', 'is_match')

scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



### Question 3

In [60]:
def tabulate(threshold):
    scored.select(expr(f'score >= {threshold} as above'), 'is_match').\
        groupBy('above').\
        pivot('is_match').\
        count().\
        show()

In [61]:
for t in range(2, 5):
    print(f'threshold = {t}')
    tabulate(t)

threshold = 2
+-----+-------+-----+
|above|  false| true|
+-----+-------+-----+
| true| 596414|20931|
|false|5131787| NULL|
+-----+-------+-----+

threshold = 3
+-----+-------+-----+
|above|  false| true|
+-----+-------+-----+
| true| 315213|20916|
|false|5412988|   15|
+-----+-------+-----+

threshold = 4
+-----+-------+-----+
|above|  false| true|
+-----+-------+-----+
| true|    637|20871|
|false|5727564|   60|
+-----+-------+-----+

